In [20]:
# Dependencies and Setup
import pandas as pd
import numpy as np


# Open CSV
gdp_data="GDP2_file.csv"

# Read files
gdp_data = pd.read_csv(gdp_data, thousands=',')
gdp_data.head()


,Concepto,T1-2010,T2-2010,T3-2010,T4-2010,T1-2011,T2-2011,T3-2011,T4-2011,T1-2012,...,T3-2016,T4-2016,T1-2017,T2-2017,T3-2017,T4-2017,T1-2018,T2-2018,T3-2018,T4-2018
0,Producto interno bruto,12803326.7,13182931.65,12954912.31,13305703.42,13252637.28,13732337.64,13438257.15,13872028.16,13354792.35,...,14231940.4,14882949.24,14371712.1,14998393.28,14921450,15499623.4,14902786.4,15413082.34,15526005.35,16139460.31


# GDP

In [36]:
#Transform Data
df = pd.melt(gdp_data,
                            id_vars=["Concepto"],
                            var_name = "Period",
                            value_name="GDP Index")
  

df ["Year"]= df ["Period"].str.slice(start=3)
df ["Quarter"]= df ["Period"].str.slice(stop=2)

df = df.replace(0, np.nan)
df = df.dropna(how='all', axis=0)
df = df.replace(np.nan, 0)

df = df.rename(columns={'Period':'period','GDP Index':'gdp','Year': 'year','Quarter': 'quarter'})
del df["Concepto"]

df["quarter"].replace({"T1":1, "T2":2, "T3":3, "T4":4}, inplace=True)

dict_q ={1:[1,2,3],2:[4,5,6],3:[7,8,9],4:[10,11,12]}
monthList=[dict_q[x[1]] for x in df["quarter"].items()]
df["month"]=monthList

#List values displays into multiple rows
df=df.explode("month")

# df.dtypes

df.head()


,period,gdp,year,quarter,month
0,T1-2010,12803326.70,2010,1,1
0,T1-2010,12803326.70,2010,1,2
0,T1-2010,12803326.70,2010,1,3
1,T2-2010,13182931.65,2010,2,4
1,T2-2010,13182931.65,2010,2,5


In [34]:
dict_q ={1:[1,2,3],2:[4,5,6],3:[7,8,9],4:[10,11,12]}
monthList=[dict_q[x[1]] for x in df["quarter"].items()]
monthList

# for x in df["quarter"].items():
#     print(dict_q[x[1]])
#     monthList.append


[[1, 2, 3],
 [4, 5, 6],
 [7, 8, 9],
 [10, 11, 12],
 [1, 2, 3],
 [4, 5, 6],
 [7, 8, 9],
 [10, 11, 12],
 [1, 2, 3],
 [4, 5, 6],
 [7, 8, 9],
 [10, 11, 12],
 [1, 2, 3],
 [4, 5, 6],
 [7, 8, 9],
 [10, 11, 12],
 [1, 2, 3],
 [4, 5, 6],
 [7, 8, 9],
 [10, 11, 12],
 [1, 2, 3],
 [4, 5, 6],
 [7, 8, 9],
 [10, 11, 12],
 [1, 2, 3],
 [4, 5, 6],
 [7, 8, 9],
 [10, 11, 12],
 [1, 2, 3],
 [4, 5, 6],
 [7, 8, 9],
 [10, 11, 12],
 [1, 2, 3],
 [4, 5, 6],
 [7, 8, 9],
 [10, 11, 12]]

In [22]:
#6. Create DB Connection
from sqlalchemy import create_engine

connection_string = "postgres:Ichliebe1*@localhost:5432/Project2"
engine = create_engine(f'postgresql://{connection_string}')

# # Confirm tables
engine.table_names() 

['barrel_price', 'inflation', 'indice', 'gdp']

In [23]:
 #7. Load DF into DB
df.to_sql(name="gdp", con=engine, if_exists="append", index=False)

IntegrityError: (psycopg2.errors.NotNullViolation) el valor null para la columna «index» viola la restricción not null
DETAIL:  La fila que falla contiene (null, T1-2010, 12803326.7, 2010, T1).

[SQL: INSERT INTO gdp ("Period", "GDP Index", "Year", "Quarter") VALUES (%(Period)s, %(GDP Index)s, %(Year)s, %(Quarter)s)]
[parameters: ({'Period': 'T1-2010', 'GDP Index': 12803326.7, 'Year': '2010', 'Quarter': 'T1'}, {'Period': 'T2-2010', 'GDP Index': 13182931.65, 'Year': '2010', 'Quarter': 'T2'}, {'Period': 'T3-2010', 'GDP Index': 12954912.31, 'Year': '2010', 'Quarter': 'T3'}, {'Period': 'T4-2010', 'GDP Index': 13305703.42, 'Year': '2010', 'Quarter': 'T4'}, {'Period': 'T1-2011', 'GDP Index': 13252637.28, 'Year': '2011', 'Quarter': 'T1'}, {'Period': 'T2-2011', 'GDP Index': 13732337.64, 'Year': '2011', 'Quarter': 'T2'}, {'Period': 'T3-2011', 'GDP Index': 13438257.15, 'Year': '2011', 'Quarter': 'T3'}, {'Period': 'T4-2011', 'GDP Index': 13872028.16, 'Year': '2011', 'Quarter': 'T4'}  ... displayi